# Chapter 2. Analyzing Subgroups

# 2.1 Slicing and Dicing Data

In the previous chapter, we looked at ways to summarize and visualize _all_ the values of a variable, like age. But what if we only wanted to know the age distribution of just the male passengers on the Titanic? To answer such questions, we have to **slice** our data. That is, we have to filter the `DataFrame` to include only those rows where a variable (e.g., `sex`) is equal to a particular value (e.g., `"male"`). 

The term _slicing_ makes sense if you imagine the data as a cube, with a different variable along each dimension. Slicing this cube along any one dimension will return a subset of the original data, corresponding to a particular value of that variable.

<img src="slice1.gif" width="300"> <img src="slice2.gif" width="300"> ([source](http://www.hypertextbookshop.com/dataminingbook/public_version/contents/chapters/chapter003/section004/blue/page004.html))

In [ ]:
%matplotlib inline

import pandas as pd
pd.options.display.max_rows = 5
df = pd.read_csv("/data301/data/titanic.csv")
df

## Two Ways to Slice a DataFrame

One way to slice a `pandas` `DataFrame` is to set the slicing variable as the index and then to select the value you want using `.loc`.

So for example, if we wanted a `DataFrame` with just the male passengers, we could do:

In [ ]:
males = df.set_index("sex").loc["male"]
males

In [ ]:
males.age.plot.hist()

Another way to slice a `DataFrame` is to use a **boolean mask**. A boolean mask is simply a boolean `Series` whose index matches the index of the `DataFrame`.

To create a boolean mask, we can use the standard comparison operators `==`, `<`, `>`, and `!=` on any column in the `DataFrame`. For example, the following code produces a boolean mask that is equal to `True` for the male passengers and equal to False otherwise.

In [ ]:
df.sex == "male"

Notice how the equality operator was automatically _broadcast_ over the elements of the `Series` `df.sex`. That is, we check whether each element of `df.sex` is equal to `"male"`, resulting in a `Series` of booleans.

This boolean mask can then be passed into a `DataFrame` to obtain just the subset of rows where the mask equals `True`.

In [ ]:
df[df.sex == "male"]

Notice how the index is missing the numbers 0 and 2 because those passengers were female. We are left with a `DataFrame` where the variable `sex` only takes on one value, `"male"`.

Now we can select the age column from this `DataFrame` and plot the distribution, as we did before.

In [ ]:
df[df.sex == "male"].age.plot.hist()

A boolean mask is a kind of selector, but it is quite different from the label-based (`.loc`) and position-based (`.iloc`) selection that we encountered before. Passing a boolean mask to `.loc` is equivalent to passing the boolean mask directly to the `DataFrame`:

In [ ]:
df.loc[df.sex == "male"]

We might prefer to pass the boolean mask to `.loc` if we also wanted to select columns at the same time. For example, if we wanted to select the ages of the male passengers, we would do:

In [ ]:
df.loc[df.sex == "male", "age"]

### Speed Comparison

We've seen two ways to slice a `DataFrame`. Which way is better?

One consideration is that the first method forces you to set the index of your `DataFrame` to the variable you want to slice by. If your `DataFrame` already has a natural index, you might not want to replace that index just to slice the data.

Another consideration is speed. Let's test the runtimes of the two options by using the `%timeit` magic. (**Warning:** The cell below will take a while to run, since `timeit` will run each command multiple times and then report the mean and standard deviation of the runtimes.)

In [ ]:
%timeit df.set_index("sex").loc["male"].age.mean()
%timeit df[df.sex == "male"].age.mean()

So boolean masking is also significantly faster than re-indexing and selecting. Taking all considerations into account, boolean masking is generally the best way to slice your data.

### Working with Boolean Series

Remember that a boolean mask is a `Series` of booleans. A boolean `Series` can be regarded as a categorical variable, but it can also be treated as a quantitative variable, where the `True`s are 1s and the `False`s are 0s. For example, the following command actually produces a `Series` of 7s and 8s.

In [ ]:
(df.sex == "male") + 7

For a more realistic example of how we might use a boolean `Series` as a quantitative variable, recall how we coded binary categorical variables as 0s and 1s. This allowed us to use functions like `.sum()` and `.mean()`, that are for quantitative variables, to learn about the categorical variable, such as the _number_ of observations in category 1 and the _proportion_ of observations in category 1. (Remember that a _proportion_ is just a percentage, expressed as a number between 0 and 1.)

Boolean `Series` are also variables containing just 0s and 1s, where a 1 represents `True`. So the command

In [ ]:
(df.sex == "male").sum()

returns the _number_ of observations where `sex == "male"` and

In [ ]:
(df.sex == "male").mean()

returns the _proportion_ of observations where `sex == "male"`. Check that these answers are correct by some other method.

## Dicing

<img src="dice.jpeg">

Dicing is a generalization of slicing. With slicing, we select a single value of a single variable. With dicing, we are allowed to select multiple values of multiple variables. Dicing allows us to answer questions like, "What was the age distribution of male _survivors_ on the Titanic?" To answer this question, we have to filter our data set on two variables, `sex` and `survived`.

We can dice a `pandas` `DataFrame` using a boolean mask, as before. The only difference is in how we obtain that boolean mask. We want a boolean mask that is `True` if `sex` is `"male"` _and_ `survived` is `1`. Let's start by creating a separate boolean mask for each of these variables.

In [ ]:
df.sex == "male"

In [ ]:
df.survived == 1

We want to combine these two boolean masks into a single mask that is `True` only if the corresponding elements of _both_ masks are `True`. This can be accomplished by joining the two `Series` by the logical operator `&`.

In [ ]:
(df.sex == "male") & (df.survived == 1)

Verify for yourself that the `True` values in this `Series` correspond to observations where _both_ masks were True.

_Warning:_ Notice how we included parentheses around each boolean mask above. These parentheses turn out to be necessary because of Python's operator precedence. The command

`df.sex == "male" & df.survived == 1`

will be interpreted by Python as 

`df.sex == ("male" & df.survived) == 1`

and result in an error. Python does not know how to evaluate `("male" & df.survived)`, since the logical operator `&` is not defined for a string and a `Series`. Parentheses ensure that Python evaluates the boolean masks first and then the logical operator:

`(df.sex == "male") & (df.survived == 1)`.

Forgetting the parentheses around boolean masks is a common error. To make matters worse, the error message that you get is not very helpful for diagnosing the problem. If you don't believe me, just try running the offending command above!

Now that we have the correct boolean mask for the dicing operation, we can plot the age distribution of the male survivors.

In [ ]:
df[(df.sex == "male") & (df.survived == 1)].age.plot.hist()

Notice the peak between 0 and 10. A disproportionate number of young children survived because they were given priority.

So far, we have worked exclusively with the `&` operator. But there are actually three logical operators that can be used to modify and combine boolean masks:

- `&` means "and"
- `|` means "or"
- `~` means "not"

Like `&`, `|` and `~` operate elementwise on boolean `Series`. Examples are provided below.

In [ ]:
# male OR survived
(df.sex == "male") | (df.survived == 1)

In [ ]:
# equivalent to (df.sex != "male")
~(df.sex == "male")

Notice how we use parentheses to ensure that the boolean mask is evaluated before the logical operators.

# Exercises

Exercises 1-3 deal with the Titanic data set.

**Exercise 1.** Is there any advantage to selecting the column at the same time you apply the boolean mask? In other words, is the second option below any faster than the first?

1. `df[df.sex == "female"].age`
2. `df.loc[df.sex == "female", "age"]`

Use the `%timeit` magic to compare the runtimes of these two options.

In [ ]:
# YOUR CODE HERE

**Exercise 2.** Make a single graphic that compares the age distribution of the males who survived with the age distribution of the males who did not.

In [ ]:
# YOUR CODE HERE

**Exercise 3.** What percentage of 1st class passengers survived? What percentage of 3rd class passengers survived?

In [ ]:
# YOUR CODE HERE

Exercises 4-7 work with the Tips data set (`/data301/data/tips.csv`). The following code reads the data into a `DataFrame` called `tips` and creates a new column called `tip_percent` out of the `tip` and `total_bill` columns. This new column represents the tip as a percentage of the total bill (as a number between 0 and 1).

In [ ]:
tips = pd.read_csv("/data301/data/tips.csv")
tips["tip_percent"] = tips.tip / tips.total_bill
tips

**Exercise 4.** Calculate the average tip percentage paid by parties of 4 or more.

In [ ]:
# YOUR CODE HERE

**Exercise 5.** Make a visualization comparing the distribution of tip percentages left by males and females. How do they compare?

In [ ]:
# YOUR CODE HERE

**Exercise 6.** What is the average table size on weekdays? (_Hint:_ There are at least two ways to create the appropriate boolean mask: using the `|` logical operator and using the `.isin()` method. See if you can do it both ways.)

In [ ]:
# YOUR CODE HERE

**Exercise 7.** Calculate the average table size for each day of the week. On which day of the week does the waiter serve the largest parties, on average?

In [ ]:
# YOUR CODE HERE